In [1]:
input_path = './2-dataset/COCO'   #Just the main folder (subfolders will be processed as well)
output_path = './4-output/blurred_COCO' #will be created in code

In [11]:
import torch
# from torchvision.transforms import functional as F
# from PIL import Image, ImageDraw, ImageFilter
from PIL import Image, ImageFilter
import os

# Load YOLOv5 models
person_model_path = './1-models/Person_Detector.pt'
head_model_path = './1-models/Head_detector_fromPersonBox_yolov5.pt'


person_model = torch.hub.load('ultralytics/yolov5', 'custom', path=person_model_path)
head_model = torch.hub.load('ultralytics/yolov5', 'custom', path=head_model_path)


# Define a function to blur faces
def blur_faces(image, head_boxes,blur=False):
    blurred_image = image.copy()
    for box in head_boxes:
        x1, y1, x2, y2 = map(int, box)  # Convert to integer coordinates
        face_region = image.crop((x1, y1, x2, y2))
        if blur:
            face_region = face_region.filter(ImageFilter.BLUR)
            face_region = face_region.filter(ImageFilter.BLUR)
            blurred_face = face_region.filter(ImageFilter.BLUR)

        else:
            blurred_face = Image.new('RGB', (x2 - x1, y2 - y1), (0, 0, 0))
        blurred_image.paste(blurred_face, (x1, y1, x2, y2))
    return blurred_image

# to ignore other files in dataset and only care about images
def is_image_file(filename):
    img_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp']
    return any(filename.lower().endswith(ext) for ext in img_extensions)

# Process the dataset
input_folder = input_path
output_folder = output_path

os.makedirs(output_folder, exist_ok=True)

for root, _, files in os.walk(input_folder):
    for filename in files:
        if is_image_file(filename):
            image_path = os.path.join(root, filename)
            img = Image.open(image_path)
            
            # Step 1: Detect persons using the first model
            results_person = person_model(img)
            persons = results_person.pandas().xyxy[0]
            person_boxes = persons[persons['name'] == 'person'][['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()
            
            # Step 2: Detect heads using the second model
            head_boxes = []
            for box in person_boxes:
                x1, y1, x2, y2 = box
                person_region = img.crop((x1, y1, x2, y2))
                results_head = head_model(person_region)
                heads = results_head.pandas().xyxy[0]
                if 'head' in heads['name'].values:
                    head = heads[heads['name'] == 'head'][['xmin', 'ymin', 'xmax', 'ymax']].values[0]
                    head_boxes.append([x1 + head[0], y1 + head[1], x1 + head[2], y1 + head[3]])
            
            # Step 3: Save YOLO format .txt file with head bounding boxes
            rel_image_path = os.path.relpath(image_path, input_folder)
            txt_output_path = os.path.join(output_folder, os.path.splitext(rel_image_path)[0] + '.txt')
            os.makedirs(os.path.dirname(txt_output_path), exist_ok=True)
            
            with open(txt_output_path, 'w') as f:
                for box in head_boxes:
                    f.write(f"head {box[0]} {box[1]} {box[2]} {box[3]}\n")
            
            # Step 4: Save the new dataset with blurred faces
            rel_output_folder = os.path.relpath(output_folder, input_folder)
            output_image_path = os.path.join(output_folder, rel_image_path)
            os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
            
            # If blur = False it will put a black box instead of head.
            # set blur = True if you do not like the black box!
            blurred_image = blur_faces(img, head_boxes, blur = False)
            blurred_image.save(output_image_path)

Using cache found in C:\Users\noor/.cache\torch\hub\ultralytics_yolov5_master
WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ------------------------------------ 188.5/188.5 kB 518.2 kB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB 1.1 MB/s eta 0:00:00

requirements: AutoUpdate success  6.7s, installed 1 package: ['gitpython>=3.1.30']
requirements:  Restart runtime or rerun command for updat